# Chrome AI Built-in Prompt API Demo

This notebook demonstrates using Chrome's built-in AI (Gemini Nano) with the Prompt API.

**Requirements:**
- Chrome browser version 128+ 
- Enable experimental flags:
  - `chrome://flags/#prompt-api-for-gemini-nano`
  - `chrome://flags/#optimization-guide-on-device-model`
- First use requires downloading the Gemini Nano model

In [ ]:
import micropip
await micropip.install("anywidget")

In [ ]:
import anywidget
import traitlets

class ChromeAIWidget(anywidget.AnyWidget):
    _esm = """
    async function render({ model, el }) {
      // Create container
      const container = document.createElement("div");
      container.className = "chrome-ai-container";
      
      // Create status message
      const status = document.createElement("div");
      status.className = "status-message";
      
      // Create input box
      const input = document.createElement("textarea");
      input.className = "prompt-input";
      input.placeholder = "Enter your prompt here...";
      input.rows = 3;
      
      // Create submit button
      const button = document.createElement("button");
      button.className = "submit-button";
      button.textContent = "Submit";
      
      // Create response box
      const response = document.createElement("div");
      response.className = "response-box";
      
      // Append elements
      container.appendChild(status);
      container.appendChild(input);
      container.appendChild(button);
      container.appendChild(response);
      el.appendChild(container);
      
      // Initialize Chrome AI session
      let session = null;
      
      // Sync input with traitlet
      input.value = model.get("input_text");
      input.addEventListener("input", () => {
        model.set("input_text", input.value);
        model.save_changes();
      });
      
      // Update input when traitlet changes from Python
      model.on("change:input_text", () => {
        input.value = model.get("input_text");
      });
      
      // Update response when traitlet changes
      const updateResponse = () => {
        response.textContent = model.get("response");
      };
      updateResponse();
      model.on("change:response", updateResponse);
      
      // Update status when traitlet changes
      const updateStatus = () => {
        const statusText = model.get("status");
        status.textContent = statusText;
        
        // Set color based on status
        if (statusText.includes("Error") || statusText.includes("not available")) {
          status.style.color = "#dc2626";
        } else if (statusText.includes("Generating") || statusText.includes("Creating")) {
          status.style.color = "#2563eb";
        } else if (statusText.includes("Ready") || statusText.includes("received")) {
          status.style.color = "#16a34a";
        } else {
          status.style.color = "inherit";
        }
      };
      updateStatus();
      model.on("change:status", updateStatus);
      
      // Initialize AI session
      (async () => {
        try {
          model.set("status", "Initializing...");
          model.save_changes();
          
          // Check if AI is available
          if (!window.ai || !window.ai.languageModel) {
            model.set("status", "Chrome AI not available. Please enable chrome://flags/#prompt-api-for-gemini-nano");
            model.save_changes();
            button.disabled = true;
            return;
          }
          
          // Create session
          model.set("status", "Creating AI session...");
          model.save_changes();
          session = await window.ai.languageModel.create();
          model.set("status", "Ready! Enter a prompt and press Submit or Enter.");
          model.save_changes();
          
        } catch (error) {
          model.set("status", `Error: ${error.message}`);
          model.save_changes();
          button.disabled = true;
        }
      })();
      
      // Handle prompt submission
      async function submitPrompt() {
        const promptText = input.value.trim();
        if (!promptText || !session) return;
        
        try {
          button.disabled = true;
          input.disabled = true;
          
          model.set("status", "Generating response...");
          model.set("prompt", promptText);
          model.set("response", "");
          model.save_changes();
          
          // Get AI response
          const result = await session.prompt(promptText);
          
          // Update traitlets
          model.set("response", result);
          model.set("status", "Response received! Enter another prompt.");
          model.save_changes();
          
        } catch (error) {
          model.set("response", `Error: ${error.message}`);
          model.set("status", "Error occurred. Please try again.");
          model.save_changes();
        } finally {
          button.disabled = false;
          input.disabled = false;
        }
      }
      
      // Button click handler
      button.addEventListener("click", submitPrompt);
      
      // Enter key handler
      input.addEventListener("keydown", (e) => {
        if (e.key === "Enter" && !e.shiftKey) {
          e.preventDefault();
          submitPrompt();
        }
      });
    }
    export default { render };
    """
    
    _css = """
    .chrome-ai-container {
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      max-width: 800px;
      margin: 0 auto;
      padding: 20px;
    }
    
    .status-message {
      padding: 12px;
      margin-bottom: 16px;
      border-radius: 6px;
      background-color: #f3f4f6;
      font-size: 14px;
      font-weight: 500;
    }
    
    .prompt-input {
      width: 100%;
      padding: 12px;
      margin-bottom: 12px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      font-size: 14px;
      font-family: inherit;
      resize: vertical;
      box-sizing: border-box;
    }
    
    .prompt-input:focus {
      outline: none;
      border-color: #3b82f6;
    }
    
    .submit-button {
      padding: 10px 20px;
      margin-bottom: 16px;
      background-color: #3b82f6;
      color: white;
      border: none;
      border-radius: 6px;
      font-size: 14px;
      font-weight: 500;
      cursor: pointer;
      transition: background-color 0.2s;
    }
    
    .submit-button:hover:not(:disabled) {
      background-color: #2563eb;
    }
    
    .submit-button:disabled {
      background-color: #9ca3af;
      cursor: not-allowed;
    }
    
    .response-box {
      padding: 16px;
      min-height: 100px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      background-color: #ffffff;
      white-space: pre-wrap;
      word-wrap: break-word;
      font-size: 14px;
      line-height: 1.6;
    }
    """
    
    input_text = traitlets.Unicode("").tag(sync=True)
    prompt = traitlets.Unicode("").tag(sync=True)
    response = traitlets.Unicode("Response will appear here...").tag(sync=True)
    status = traitlets.Unicode("Initializing...").tag(sync=True)

## Create and Display the Widget

Create an instance of the Chrome AI widget and display it:

In [ ]:
ai_widget = ChromeAIWidget()
ai_widget

## Access Widget State from Python

The widget uses traitlets to synchronize state between JavaScript and Python. You can access all widget properties:

In [ ]:
print("Input text:", ai_widget.input_text)
print("Last prompt:", ai_widget.prompt)
print("Last response:", ai_widget.response)
print("Status:", ai_widget.status)